# sPyTial Structured Input Demo

This notebook demonstrates the new structured input functionality in sPyTial, which allows users to:

- **Create data structures with "holes"** - placeholders that can be filled interactively
- **Use templates** - predefined patterns for common data structures
- **Interactive REPL** - build data structures step by step with live visualization
- **Direct manipulation** - modify data through both command interface and visual feedback

This addresses the need for structured input capabilities as outlined in the original issue.

In [ ]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

import spytial
from spytial import (
    structured_input, 
    Hole, 
    create_hole, 
    create_template,
    start_from_template,
    fill_hole_by_description,
    get_current_state,
    get_result,
    list_templates,
    reset
)
import json

## 1. Basic Holes and Templates

Let's start by exploring the concept of "holes" - placeholders in data structures that can be filled later.

In [ ]:
# Create a hole - a placeholder for a value
name_hole = create_hole("str", "person_name")
age_hole = create_hole("int", "person_age")

print("Created holes:")
print(f"Name hole: {name_hole}")
print(f"Age hole: {age_hole}")

# Create a data structure with holes
person_template = {
    "name": name_hole,
    "age": age_hole,
    "status": "active",  # Pre-filled value
    "metadata": {
        "created": "2024-01-01",
        "last_updated": create_hole("str", "update_time")
    }
}

print("\nPerson template with holes:")
print(json.dumps(person_template, indent=2, default=str))

## 2. Using the Template System

sPyTial provides a template system with common data structure patterns pre-built.

In [ ]:
# List available templates
print("Available templates:")
templates = list_templates()
for template_name in templates:
    print(f"  - {template_name}")

# Start with a simple dictionary template
print("\n=== Working with simple_dict template ===")
template = start_from_template("simple_dict")
print(f"Started template: {template.name}")
print(f"Description: {template.description}")

# Check the current state
state = get_current_state()
print("\nCurrent state:")
print(json.dumps(state, indent=2))

In [ ]:
# Fill the hole in our template
print("Filling the 'value' hole with 'Hello, World!'")
updated_template = fill_hole_by_description("value", "Hello, World!")

# Check if the template is complete
print(f"\nTemplate complete: {updated_template.is_complete()}")

if updated_template.is_complete():
    result = get_result()
    print("\nFinal result:")
    print(json.dumps(result, indent=2))
    
    # Visualize the result
    print("\nVisualizing the completed structure:")
    spytial.diagram(result)

## 3. Building Complex Structures - Tree Example

Let's work with a more complex template - a binary tree node.

In [ ]:
# Reset and start with tree template
reset()
print("=== Building a Tree Structure ===")

tree_template = start_from_template("tree_node")
print(f"Started with: {tree_template.name}")

# Show the holes we need to fill
state = get_current_state()
print("\nHoles to fill:")
for hole in state['holes']:
    print(f"  - {hole['description']} ({hole['type_hint']})")

# Fill the root value
print("\nFilling root value...")
fill_hole_by_description("node_value", "Root")

# For tree nodes, we can create nested structures
print("Creating left child...")
left_child = {"value": "Left Child", "left": None, "right": None}
fill_hole_by_description("left_child", left_child)

print("Creating right child...")
right_child = {"value": "Right Child", "left": None, "right": None}
fill_hole_by_description("right_child", right_child)

# Get the final tree
tree = get_result()
print("\nCompleted tree structure:")
print(json.dumps(tree, indent=2))

In [ ]:
# Visualize the tree with spatial annotations
print("Visualizing the tree structure:")

# Apply spatial constraints to make it look like a tree
spytial.annotate_orientation(tree, "value", ["above"])
spytial.diagram(tree)

## 4. Interactive Structured Input REPL

The most powerful feature is the interactive REPL interface that allows real-time building and visualization of data structures.

In [ ]:
# Launch the interactive structured input interface
print("Launching interactive structured input interface...")
print("This will open an interface where you can:")
print("  - Select templates from the left panel")
print("  - Fill holes using REPL commands")
print("  - See live visualization updates")
print("  - Build complex structures step by step")
print("\nTry these commands in the REPL:")
print("  start simple_list")
print("  fill first_item 42")
print("  fill second_item \"hello\"")
print("  result")

# This will open in a new browser tab or inline depending on environment
structured_input()

## 5. Creating Custom Templates

You can also create your own templates for specific use cases.

In [ ]:
# Create a custom template for a student record
print("=== Creating Custom Student Record Template ===")

student_name = create_hole("str", "student_name")
student_id = create_hole("int", "student_id")
grades = create_hole("list", "grades_list")
advisor = create_hole("str", "advisor_name")

student_template = create_template(
    "student_record",
    {
        "personal": {
            "name": student_name,
            "id": student_id,
            "year": "2024"
        },
        "academic": {
            "grades": grades,
            "advisor": advisor,
            "status": "enrolled"
        }
    },
    "Student academic record template"
)

print(f"Created template: {student_template.name}")
print(f"Holes to fill: {len(student_template.holes)}")

# Register it with the default builder
spytial.default_builder.register_template(
    "student_record", 
    student_template.template, 
    student_template.description,
    student_template.holes
)

print("\nRegistered with builder. Available templates now:")
for template_name in list_templates():
    print(f"  - {template_name}")

In [ ]:
# Use the custom template
reset()
template = start_from_template("student_record")

# Fill in the student information
fill_hole_by_description("student_name", "Alice Johnson")
fill_hole_by_description("student_id", 12345)
fill_hole_by_description("grades_list", ["A", "B+", "A-", "B"])
fill_hole_by_description("advisor_name", "Dr. Smith")

# Get the completed record
student_record = get_result()
print("Completed student record:")
print(json.dumps(student_record, indent=2))

# Visualize the structure
print("\nVisualizing student record:")
spytial.diagram(student_record)

## 6. Advanced: Live REPL with Custom Data

The structured input interface can also start with existing data and allow modifications.

In [ ]:
# Start the structured input with some initial data
initial_graph = {
    "nodes": [
        {"id": "A", "value": 1},
        {"id": "B", "value": 2},
        {"id": "C", "value": 3}
    ],
    "edges": [
        {"from": "A", "to": "B"},
        {"from": "B", "to": "C"}
    ]
}

print("Launching structured input with initial graph data...")
print("You can now:")
print("  - Use templates to add new structures")
print("  - See the existing data visualized")
print("  - Build on top of the existing structure")

structured_input(initial_data=initial_graph, method="file")

## Conclusion

The structured input functionality provides a powerful way to:

1. **Build data incrementally** using holes and templates
2. **Interactive construction** through the REPL interface
3. **Live visualization** of data as it's being built
4. **Template system** for common patterns
5. **Custom templates** for domain-specific needs

This addresses the original issue by providing:
- ✅ Structured input capabilities
- ✅ Interactive REPL with live view
- ✅ Support for constructing values with "holes"
- ✅ Direct manipulation through commands
- ✅ Foundation for visual editing (extendable)

The system is designed to be extensible, allowing for future enhancements like:
- Visual drag-and-drop editing
- More sophisticated constraint systems
- Integration with external data sources
- Custom visualization patterns for templates